In [1]:
import os
import requests
from bs4 import BeautifulSoup
import urllib.parse
import pandas as pd

In [2]:
session = requests.Session()

In [3]:
list_url = "https://wiki.biligame.com/whmx/%E5%99%A8%E8%80%85%E5%9B%BE%E9%89%B4"
list_html = session.get(list_url).text
list_soup = BeautifulSoup(list_html, "lxml")
role_html_list = list_soup.find("div", id="CardSelectTr").find_all("div", class_="visible-xs")
for role_html in role_html_list:
    role_html = role_html.find("a")
    role_url = "https://wiki.biligame.com" + role_html["href"]
    role_name = role_url.split("/")[-1]
    role_url = role_url.replace(role_name, f"index.php?title={role_name}&action=edit")
    role_name = urllib.parse.unquote(role_name)
    if os.path.exists(f"./data/txt/{role_name}.txt"):
        continue
    role_info_html = session.get(role_url).text
    role_info_soup = BeautifulSoup(role_info_html, "lxml")
    role_info = role_info_soup.find("textarea").text
    with open(f"./data/txt/{role_name}.txt", "w+", encoding="utf-8") as f:
        f.write(role_info)

In [4]:
columns = ["器者名称", "稀有度", "职业", "生命值", "生命值最大", "攻击", "攻击最大", "物理防御", "物理防御最大", "构素防御", "构素防御最大", "速度", "移动力", "暴击率", "暴击伤害", "暴击抵抗", "闪避率", "格挡率", "格挡强度", "水平跳跃", "垂直跳跃", "初始能量", "能量上限", "治疗量提升", "受治疗提升", "生命偷取率", "瞄准冷却率", "能量自动获取", "防御穿透", "贯穿概率", "最大礼弹上限", "额外装填礼弹", "命中率", ]
data = pd.DataFrame(columns=columns)
data

,器者名称,稀有度,职业,生命值,生命值最大,攻击,攻击最大,物理防御,物理防御最大,构素防御,...,治疗量提升,受治疗提升,生命偷取率,瞄准冷却率,能量自动获取,防御穿透,贯穿概率,最大礼弹上限,额外装填礼弹,命中率


In [5]:
def parse_role_info(text):
    result = {
        "暴击率": "0",
        "暴击伤害": "150",
        "暴击抵抗": "0",
        "闪避率": "0",
        "格挡率": "0",
        "格挡强度": "0",
        "水平跳跃": "0",
        "垂直跳跃": "1",
        "初始能量": "1",
        "治疗量提升": "0",
        "受治疗提升": "0",
        "生命偷取率": "0",
        "瞄准冷却率": "0",
        "能量自动获取": "0",
        "防御穿透": "0",
        "贯穿概率": "0",
        "最大礼弹上限": "0",
        "额外装填礼弹": "0",
        "命中率": "100",
    }
    for line in text.split("\n"):
        for column in columns:
            if line.startswith(f"|{column}="):
                result[column] = line.split("=")[-1].replace(" ", "").replace("\t", "")
    return result

In [6]:
for txt in os.listdir("./data/txt"):
    with open(f"./data/txt/{txt}", "r", encoding="utf-8") as f:
        text = f.read()
    data = data.merge(pd.DataFrame([parse_role_info(text)]), how="outer")
data

,暴击率,暴击伤害,暴击抵抗,闪避率,格挡率,格挡强度,水平跳跃,垂直跳跃,初始能量,治疗量提升,...,生命值最大,攻击,攻击最大,物理防御,物理防御最大,构素防御,构素防御最大,速度,移动力,能量上限
0,0,150,0,0,0,0,0,1,1,0,...,6352,345,2622,39,435,39,435,120,3,8
1,0,150,0,0,0,0,1,2,1,0,...,5353,284,2165,37,433,37,433,175,4,3
2,0,150,0,0,0,0,0,1,1,0,...,4798,339,2616,33,330,33,330,95,3,5
3,0,150,0,0,0,0,1,2,1,0,...,6239,348,2625,58,652,58,652,100,4,3
4,0,150,0,0,0,0,0,1,1,0,...,5687,319,2398,33,330,33,330,80,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,0,150,0,0,0,0,1,2,1,0,...,4572,276,2058,36,432,36,432,170,4,2
75,0,150,0,0,0,0,0,1,1,0,...,6908,285,2166,44,440,44,440,180,3,3
76,0,150,0,0,0,0,0,1,1,0,...,4127,271,2053,25,223,25,223,60,3,3
77,0,150,0,0,0,0,0,1,1,0,...,5570,321,2400,32,329,32,329,145,3,3


In [7]:
# data.sort_values(by=["稀有度","职业"], inplace=True)
# data
# 在data的特定的列按特定值排序，将稀有度列按照特出、优异、新生的顺序排序
data["稀有度"] = data["稀有度"].map({"特出": 3, "优异": 2, "新生": 1})
data.sort_values(by=["稀有度", "职业"], ascending=[False, True], inplace=True)
data["稀有度"] = data["稀有度"].map({3: "特出", 2: "优异", 1: "新生"})
data = data[columns]
data

,器者名称,稀有度,职业,生命值,生命值最大,攻击,攻击最大,物理防御,物理防御最大,构素防御,...,治疗量提升,受治疗提升,生命偷取率,瞄准冷却率,能量自动获取,防御穿透,贯穿概率,最大礼弹上限,额外装填礼弹,命中率
37,曾侯乙编钟,特出,宿卫,1620,8055,300,2280,42,438,42,...,0,0,0,0,0,0,0,0,0,100
41,毛公鼎,特出,宿卫,1770,8799,278,2159,58,652,58,...,0,0,0,0,0,0,0,0,0,100
43,溪山行旅图,特出,宿卫,1800,8928,265,2047,59,653,59,...,0,0,0,0,0,0,0,0,0,100
68,鎏金骑士,特出,宿卫,1696,8428,300,2280,48,543,48,...,0,0,0,0,0,0,0,0,0,100
69,铜奔马,特出,宿卫,1650,8184,286,2167,43,439,43,...,0,0,0,0,0,0,0,0,0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,蛙锣,新生,轻锐,637,5686,228,1713,38,434,38,...,0,0,0,0,0,0,0,0,0,100
74,驿使图砖,新生,轻锐,513,4572,276,2058,36,432,36,...,0,0,0,0,0,0,0,0,0,100
15,可爱花束,新生,远击,610,5461,251,1934,25,322,25,...,0,0,0,0,0,0,0,0,0,100
31,山水人物镜,新生,远击,567,5022,251,1934,25,223,25,...,0,0,0,0,0,0,0,0,0,100


In [8]:
data.to_csv("./data/role_info.csv", index=False, encoding="utf-8-sig")

In [10]:
# save to sqlite
import sqlite3
conn = sqlite3.connect("./data/role_info.db")
data.to_sql("role_info", conn, if_exists="replace", index=False)
conn.close()